In [76]:
import numpy as np
import umap
import umap.plot
import matplotlib.pyplot as plt
import pandas as pd

In [77]:
ids, embeddings, species, descriptions, has_wings = np.load('../../short/short_protein_id_lookup.npz').values()

In [101]:
N = 10000
embedding_subset = embeddings[:N]
has_wings_subset = has_wings[:N]
species_subset = species[:N]
descriptions_subset = descriptions[:N]

## UMAP

In [102]:
mapper = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
mapper.fit(embedding_subset)
# umap.plot.points(mapper.fit(embedding_subset), labels=has_wings_subset)

plt.show()

/home/alex/.local/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [110]:
from bokeh.io import output_notebook, show

hover_data = pd.DataFrame({
    # 'species': species_subset,
    'description': descriptions_subset,
})
output_notebook()
p = umap.plot.interactive(mapper, hover_data=hover_data, labels=has_wings_subset, color_key_cmap='plasma')
umap.plot.show(p)

Loading BokehJS ...

## Random Forest

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# It's good practice to hold out a test set
X_train, X_test, y_train, y_test = train_test_split(
    embeddings, has_wings, test_size=0.2
)

# Initialize the model
# n_jobs=-1 uses all CPU cores
# max_depth limits tree size to prevent overfitting and speed up training
rf = RandomForestClassifier(n_estimators=100, max_depth=30, n_jobs=4)

print("Training model...")
rf.fit(X_train, y_train)

print("Model Accuracy:", rf.score(X_test, y_test))

Training model...
Model Accuracy: 0.6520973083128131


In [31]:
# Get the importance scores from the trained model
importances = rf.feature_importances_

# Assume you have a list of your feature names
# e.g., ['domain_A', 'domain_B', 'length', 'charge', ...]

# Create a DataFrame to see the most important features
importance_df = pd.DataFrame({
    # 'feature': descriptions,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(importance_df.head(20))

    importance
2     0.031955
1     0.031756
13    0.031744
25    0.031718
11    0.031657
6     0.031639
18    0.031619
8     0.031345
23    0.031345
16    0.031337
27    0.031314
7     0.031290
10    0.031282
9     0.031268
4     0.031254
3     0.031214
29    0.031210
22    0.031149
31    0.031142
17    0.031124


In [49]:
probablities = rf.predict_proba(embedding_subset)

candidates = np.where((probablities[:,1] > 0.99) & (has_wings_subset == 1))[0]

In [51]:
candidate_descriptions = descriptions[candidates]
candidate_descriptions = [desc.split('isoform')[0].strip() for desc in candidate_descriptions]
print('\n'.join(set(candidate_descriptions)))

F-box only protein 34
MAP kinase-activating death domain protein
pericentriolar material 1 protein
tubulin polyglutamylase TTLL5
olfactory receptor 13-like
regulator of telomere elongation helicase 1
syntabulin
disintegrin and metalloproteinase domain-containing protein 15
patr class I histocompatibility antigen, B-2 alpha chain-like
olfactory receptor 5B3-like
methyl-CpG-binding domain protein 1
dystonin
single-stranded DNA-binding protein 4
nuclear receptor corepressor 2
bromodomain adjacent to zinc finger domain protein 2B
guanine nucleotide exchange factor DBS


## Clustering

In [72]:
from sklearn.cluster import MiniBatchKMeans, AgglomerativeClustering

n_micro_clusters = 10000
micro_model = MiniBatchKMeans(n_clusters=n_micro_clusters, batch_size=10000)
micro_labels = micro_model.fit_predict(embeddings) # X is your (3M, 32) array
centroids = micro_model.cluster_centers_

In [ ]:
df = pd.DataFrame({
    'ids': ids,
    'species': species,
    'description': descriptions,
    'has_wings': has_wings,
    'micro_cluster': micro_labels
})
df.groupby('micro_cluster').head()


,ids,species,description,has_wings,micro_cluster
0,XP_028357963.1,Phyllostomus discolor,transmembrane protein 231 isoform X1,True,14
1,XP_008137899.1,Eptesicus fuscus,transmembrane protein 231 isoform X2,True,14
2,XP_019566359.2,Rhinolophus sinicus,transmembrane protein 231,True,14
3,XP_004659664.2,Jaculus jaculus,transmembrane protein 231,False,14
4,XP_059118539.1,Peromyscus eremicus,transmembrane protein 231,False,14
...,...,...,...,...,...
129906,XP_074182058.1,Rhinolophus sinicus,pleckstrin homology domain-containing family A...,True,317
129907,XP_074182057.1,Rhinolophus sinicus,pleckstrin homology domain-containing family A...,True,317
129908,XP_019570202.2,Rhinolophus sinicus,pleckstrin homology domain-containing family A...,True,317
129909,XP_019570193.2,Rhinolophus sinicus,pleckstrin homology domain-containing family A...,True,317


In [74]:
for label in set(micro_labels):
    cluster_members = df[df['micro_cluster'] == label]
    winged_count = cluster_members['has_wings'].sum()
    total_count = len(cluster_members)
    if winged_count == 0 or total_count == 0:
        continue
    if winged_count / total_count > 0.7:
        candidate_descriptions = cluster_members['description'].values
        candidate_descriptions = [desc.split('isoform')[0].strip() for desc in candidate_descriptions]
        print('\n'.join(set(candidate_descriptions)))
        print(f"Micro-cluster {label}: {winged_count}/{total_count} have wings")

LOW QUALITY PROTEIN: 26S proteasome non-ATPase regulatory subunit 7-like
mevalonate kinase
integrator complex subunit 4
eukaryotic translation initiation factor 2-alpha kinase 3
ATP-binding cassette sub-family D member 1
paraneoplastic antigen-like protein 8C
26S proteasome non-ATPase regulatory subunit 7
pre-B-cell leukemia transcription factor-interacting protein 1
microtubule-associated protein 2
myosin light chain kinase 3
low-density lipoprotein receptor-related protein 4
exosome complex component CSL4
apolipoprotein M
ubiquitin carboxyl-terminal hydrolase 17-like protein B
LOW QUALITY PROTEIN: mevalonate kinase
arginase-1
oxidative stress-responsive serine-rich protein 1
transmembrane protein 238-like
calmin
ester hydrolase C11orf54 homolog
mas-related G-protein coupled receptor member D
Micro-cluster 466: 152/203 have wings
